<a href="https://colab.research.google.com/github/MamatorHack/MiroThinker-GG-COLAB/blob/main/MiroThinker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os, sys, subprocess, textwrap, platform, torch

print("Python:", sys.version)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
else:
    print("⚠️ Active bien le GPU dans Colab: Exécution > Modifier le type d’exécution > GPU")

# Versions utiles (optionnel)
!nvidia-smi || true

Python: 3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]
CUDA available: True
GPU: Tesla T4
Thu Aug 14 09:14:23 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   67C    P8             10W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |   

In [2]:
# Install de base (Transformers + Accelerate + bitsandbytes pour 4-bit)
!pip -q install --upgrade pip
!pip -q install "transformers>=4.43.0" "accelerate>=0.33.0" sentencepiece
!pip -q install bitsandbytes

# (Optionnel mais conseillé en Colab pour stabilité CUDA/BBNB)
!pip -q install triton

import torch, transformers, bitsandbytes as bnb
print("Transformers:", transformers.__version__)
print("Torch:", torch.__version__)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 33.4 MB/s eta 0:00:00
Transformers: 4.55.0
Torch: 2.6.0+cu124


In [3]:
# 👇 Choisis ici :
MODEL_ID = "miromind-ai/MiroThinker-8B-SFT-v0.1"   # léger, recommandé T4
# MODEL_ID = "miromind-ai/MiroThinker-14B-SFT-v0.1"  # plus lourd, ok en 4-bit sur T4 (selon charge)

print("Model:", MODEL_ID)

Model: miromind-ai/MiroThinker-8B-SFT-v0.1


In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,        # 4-bit
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tok = AutoTokenizer.from_pretrained(
    MODEL_ID,
    use_fast=True,
    token=os.environ.get("HF_TOKEN") if os.environ.get("HF_TOKEN") else None
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto",
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    token=os.environ.get("HF_TOKEN") if os.environ.get("HF_TOKEN") else None
)

print("✅ Modèle chargé")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

model-00004-of-00005.safetensors:   0%|          | 0.00/3.19G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

✅ Modèle chargé
